In [23]:
import numpy as np # linear algebra
import pandas as pd # pandas for dataframe based data processing and CSV file I/O

In [24]:
import requests # for http requests
from bs4 import BeautifulSoup # for html parsing and scraping
from fastnumbers import isfloat 
from fastnumbers import fast_float
from multiprocessing.dummy import Pool as ThreadPool 
import bs4

import matplotlib.pyplot as plt
import seaborn as sns
import json
from tidylib import tidy_document # for tidying incorrect html

sns.set_style('whitegrid')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [25]:

def ffloat(string):
    if string is None:
        return np.nan
    if type(string)==float or type(string)==np.float64:
        return string
    if type(string)==int or type(string)==np.int64:
        return string
    return fast_float(string.split(" ")[0].replace(',','').replace('%',''),
                      default=np.nan)

def ffloat_list(string_list):
    return list(map(ffloat,string_list))

def remove_multiple_spaces(string):
    if type(string)==str:
        return ' '.join(string.split())
    return string

In [26]:
url1="https://www.bseindia.com/corporates/Forth_Results.aspx"
url="https://www.nseindia.com/companies-listing/corporate-filings-event-calendar"
url2= "https://economictimes.indiatimes.com/markets/stocks/mcalendar.cms"


headers = {
    'authority': 'beta.nseindia.com',
    'cache-control': 'max-age=0',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
    'sec-fetch-user': '?1',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
}


In [27]:
'''
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, '
                         'like Gecko) '
                         'Chrome/80.0.3987.149 Safari/537.36',
           'accept-language': 'en,gu;q=0.9,hi;q=0.8', 'accept-encoding': 'gzip, deflate, br'}
'''
session = requests.Session()
request = session.get(url, headers=headers, timeout=5)
cookies = dict(request.cookies)
response = session.get(url, headers=headers, timeout=5, cookies=cookies)
print(response.status_code)
print(response.content)

"\nheaders = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, '\n                         'like Gecko) '\n                         'Chrome/80.0.3987.149 Safari/537.36',\n           'accept-language': 'en,gu;q=0.9,hi;q=0.8', 'accept-encoding': 'gzip, deflate, br'}\n"

200
b'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" />\n<title>Corporate Filings Event Calendar - Equity, SME - NSE India</title>\n<meta name="description" content="Corporate Filings Event Calendar -  Get latest information about Corporate Filings Event Calendar - Equity, SME and more. Learn more about Corporate Filings Event Calendar Today, visit NSE India." />\n<meta name="keywords" content="" />\n<meta name="robots" content="index,follow" />\n<link rel="shortcut icon" href="/assets/images/favicon.ico" type="image/x-icon" />\n<meta property="og:title" content="" />\n<meta property="og:description" content="" />\n<meta property="og:image" content="" />\n<meta property="o

In [28]:
soup = BeautifulSoup(response.content,"html.parser")

In [29]:
select_object=soup.find('select',{'id':'eventCalender_equities_fourthComing'})
print(select_object)

<select class="forthComingSelect" id="eventCalender_equities_fourthComing">
<option value="">All Forthcoming</option>
<option value="0D">Today</option>
<option data-date="forward" value="7D">Next 1 Week</option>
<option data-date="forward" value="15D">Next 15 Days</option>
<option data-date="forward" value="1M">Next 1 Month</option>
<option data-date="forward" value="3M">Next 3 Months</option>
</select>


In [30]:
#new_tag = BeautifulSoup('<option value="15D" data-date="forward">Next 15 Days</option>').option
#<option value="">All Forthcoming</option>

for item in select_object:
    if str(item) == '<option value="">All Forthcoming</option>':
        del item['selected']
    if str(item) == '<option value="15D" data-date="forward">Next 15 Days</option>':
        item['selected'] = 'selected'

response = session.post(url, params=submit_data,timeout=15 , headers=headers)

soup = BeautifulSoup(response.content)

        
submit_data = str(select_object).replace("forthComingSelect","forthComingSelect active")       
#<select class="forthComingSelect active" id="eventCalender_equities_fourthComing">

print(soup)

<select class="forthComingSelect active" id="eventCalender_equities_fourthComing">
<option value="">All Forthcoming</option>
<option value="0D">Today</option>
<option data-date="forward" value="7D">Next 1 Week</option>
<option data-date="forward" value="15D">Next 15 Days</option>
<option data-date="forward" value="1M">Next 1 Month</option>
<option data-date="forward" value="3M">Next 3 Months</option>
</select>
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Resource not found</title>
</head>
<body style="background-color: #e8e8e8;">
<div style="font-size: 32px; padding: 50px 25px; text-align: center;">
    Resource not found
  </div>
</body>
</html>



In [9]:
'''
action = soup.find('div', 'custom_select').get('action')
post_url = get_post_url(url, action)

# parse the html and prepare the form
form = {'limit': '0', ...}

# send post request the form data
response = requests.post(post_url, data=form)

'''

"\naction = soup.find('div', 'custom_select').get('action')\npost_url = get_post_url(url, action)\n\n# parse the html and prepare the form\nform = {'limit': '0', ...}\n\n# send post request the form data\nresponse = requests.post(post_url, data=form)\n\n"

In [10]:
'''
request=requests.post(url,params=str(select _object), headers=headers)
soup=BeautifulSoup(request.content)
'''

'\nrequest=requests.post(url,params=str(select _object), headers=headers)\nsoup=BeautifulSoup(request.content)\n'

In [12]:


response = session.post(url, data=submit_data,timeout=15 , headers=headers)

soup = BeautifulSoup(response.content)

'''
for row in soup.select('table.tData01 tr'):
    print [td.text for td in row.find_all('td')]
'''

"\nfor row in soup.select('table.tData01 tr'):\n    print [td.text for td in row.find_all('td')]\n"

In [13]:
print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Resource not found</title>
</head>
<body style="background-color: #e8e8e8;">
<div style="font-size: 32px; padding: 50px 25px; text-align: center;">
    Resource not found
  </div>
</body>
</html>

